In [1]:
#import necessary dependencies
import pandas as pd
import numpy as np
import os
import boto3
from botocore.exceptions import NoCredentialsError
import io
from io import StringIO
from dotenv import load_dotenv
import traceback
import sys
from sqlalchemy import create_engine
import configparser
from langdetect import detect
import langdetect.lang_detect_exception
from deep_translator import GoogleTranslator
import re
import numpy as np
import random

In [47]:
%store -r

In [133]:
%store raw_data

Stored 'raw_data' (dict)


In [ ]:
%load_ext sql

In [ ]:
config = configparser.ConfigParser()
config.read('.env')

In [ ]:
aws_secret_key = config['AWS']['secret_key']
aws_access_key = config['AWS']['access_key']





In [3]:
#upload raw_data
raw_dubai_data = pd.read_csv("airbnb_listings/DubaiData.csv")
raw_la_data = pd.read_csv("airbnb_listings/LAData.csv")
raw_london_data = pd.read_csv("airbnb_listings/LondonData.csv")
raw_miami_data = pd.read_csv("airbnb_listings/MiamiData.csv")
raw_nyc_data = pd.read_csv("airbnb_listings/NYCData.csv")
raw_sanfransisco_data = pd.read_csv("airbnb_listings/SanFransiscoData.csv")
raw_sydney_data = pd.read_csv("airbnb_listings/SydneyData.csv")
raw_tokyo_data = pd.read_csv("airbnb_listings/TokyoData.csv")
raw_toronto_data = pd.read_csv("airbnb_listings/TorontoData.csv")

In [4]:
#make a copy of the raw_data
dubaiData = raw_dubai_data.copy()
losAngelesData =  raw_la_data.copy()
londonData  = raw_london_data.copy()
miamiData = raw_miami_data.copy()
newYorkCityData = raw_nyc_data.copy()
sanFranciscoData = raw_sanfransisco_data.copy()
sydneyData = raw_sydney_data.copy()
tokyoData =  raw_tokyo_data.copy()
torontoData  = raw_toronto_data.copy()

In [107]:
#store all dfs in a dictionary
raw_data = {
    "dubaiData": dubaiData,
    "losAngelesData": losAngelesData,
    "londonData": londonData,
    "miamiData": miamiData,
    "newYorkCityData": newYorkCityData,
    "sanFranciscoData": sanFranciscoData,
    "sydneyData": sydneyData,
    "tokyoData": tokyoData,
    "torontoData": torontoData
}

In [ ]:
#upload to S3_Bucket
s3_client = boto3.client('s3',
                         aws_access_key_id = aws_access_key,
                         aws_secret_access_key =aws_secret_key
                        )

In [ ]:
response = s3_client.create_bucket(
            Bucket = 'raw_airbnb_listings_data',
            CreateBucketConfiguration = {
                'LocationConstraint': 'eu-west-2',
            },
            )
print(response)

In [ ]:
#upload to S3 bucket
def upload_to_s3(df, Bucket, object_key):
    try:
        buffer = io.BytesIO()
        df.to_csv(buffer, index = False)
        buffer.seek(0)
        s3_client.upload_fileobj(buffer, Bucket, object_key)
        print(f"file uploaded to S3://{Bucket}/{object_key}.csv")
    except NoCredentialsError:
        print("Credentials not available or incorrect")
    except Exception as e:
        print(f"Upload to S3://{Bucket}/{object_key} failed: {e}")
        

In [10]:
dubaiData.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,City,Zipcode,Currency Native,Number of Reviews,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Spacious & Private Townhome | 5 mins drive to ...,Entire rental unit,entire_home,2023-07-31,2024-01-10,AE,Dubai,25314,AED,1,...,28,337,18,77,9,6,145,532.1,3126,11472.4
1,Spacious and Ornate 2BR Apartment in Dubai South,Entire rental unit,entire_home,2023-07-31,2024-01-10,AE,Dubai,25314,AED,24,...,333,32,217,78,24,21,79,289.9,21815,80061.1
2,Exceptional 2BR near Expo City Dubai!,Entire rental unit,entire_home,2023-07-31,2024-01-10,AE,دبي,25314,AED,13,...,219,146,176,96,14,11,70,256.9,14717,54011.4
3,Stylish Apartment Surrounded by Lush Green Parks!,Entire rental unit,entire_home,2023-07-31,2024-01-10,AE,Dubai,25314,AED,12,...,253,112,182,86,9,6,192,704.6,41775,153314.2
4,OFFER! 50% OFF! I am very cool and sleep 6!,Entire condo,entire_home,2023-07-31,2024-01-03,AE,Dubai,25314,AED,10,...,137,228,50,44,6,3,74,271.6,4461,16371.9


In [46]:
losAngelesData.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,海景泳池休闲别墅,Entire villa,entire_home,2023-07-31,2023-11-01,US,California,Los Angeles,90732,USD,...,149,216,94,76,22,19,976,976,110522,110522
1,Fall Getaway @ 75% off Local Resort Price,Entire home,entire_home,2022-12-12,2023-01-04,US,California,Los Angeles,90732,USD,...,36,329,6,29,1,0,853,853,25485,25485
2,The Peninsula Beach House,Entire home,entire_home,2023-07-31,2024-01-10,US,California,Los Angeles,90732,USD,...,350,15,170,58,19,16,817,817,170126,170126
3,Peninsula ocean View Pool Villa,Entire villa,entire_home,2023-07-31,2023-11-01,US,California,Los Angeles,90732,USD,...,59,306,32,65,1,-2,834,834,31984,31984
4,无敌海景、泳池、烧烤欢乐别墅,Private room in villa,private_room,2023-07-31,2023-11-01,US,California,Los Angeles,90732,USD,...,172,193,57,40,39,36,652,652,44858,44858


In [38]:
londonData.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Twin Ensuite with Shower at Colnbrook Lodge Gu...,Private room in guesthouse,private_room,2022-07-23,2022-07-27,GB,UK,Colnbrook,SL3 0,GBP,...,0,365,0,28,0,4,120,94.8,874,690.5
1,Single Shared Bathroom at Colnbrook Lodge Gues...,Private room in guesthouse,private_room,2022-07-23,2022-08-03,GB,UK,Colnbrook,SL3 0,GBP,...,12,353,4,44,0,3,67,52.9,1120,884.8
2,Luxury New Alpine Lodge in Private Walled Garden,Entire cabin,entire_home,2023-01-18,2023-05-03,GB,UK,Claygate,KT10 0,GBP,...,143,222,36,30,8,4,204,161.2,8752,6914.1
3,Chessington World of Adventures House,Entire home,entire_home,2022-07-22,2022-08-03,GB,UK,Greater London,KT9 2,GBP,...,11,354,5,55,0,3,153,120.9,980,774.2
4,Charming house -15min to London,Private room in home,private_room,2022-12-12,2023-10-11,GB,UK,Chessington,KT9 2,GBP,...,365,0,163,54,103,93,46,36.3,9067,7162.9


In [48]:
miamiData.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Last Host Count Updated Date,Country,State,City,Zipcode,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Cozy Family home with a pool,Entire home,entire_home,2023-07-31,2024-01-03,NaN,US,NaN,Miami,33138,...,84,281,6,9,0,-3,222,222,1678,1678
1,Coconut Grove Cottage in the Heart of the Vill...,Entire guesthouse,entire_home,2023-07-31,2023-10-11,NaN,US,Florida,Miami,33133,...,178,187,83,56,21,18,94,94,11470,11470
2,DOWNTOWN MODERN INCREDIBLE VIEWS 2/2 LOFT,Entire rental unit,entire_home,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33132,...,92,273,70,91,15,12,291,291,25863,25863
3,"WELCOME TO PARADISE MIAMI, FLORIDA",Private room in rental unit,private_room,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33129,...,210,155,168,96,37,34,60,60,12096,12096
4,Garden Cottage- Heart of Design District,Entire guesthouse,entire_home,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33137,...,53,312,39,88,2,-1,52,52,2575,2575


In [49]:
newYorkCityData.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Two-Story Condo 4 Houses to the Beach in Holgate,Entire condo,entire_home,2022-12-12,2023-03-16,US,New Jersey,Long Beach Township,8008.0,USD,...,33,332,7,25,0,-3,350,350,2888,2888
1,Spectacular Oceanfront Beach Escape,Entire home,entire_home,2023-07-31,2023-12-13,US,New Jersey,Long Beach,8008.0,USD,...,30,335,8,32,0,-3,2627,2627,25219,25219
2,Adventure Cottage | Near LBI | Million Dollar ...,Entire home,entire_home,2023-07-31,2023-08-16,US,New Jersey,Stafford Township,8092.0,USD,...,106,259,55,60,16,15,232,232,25507,25507
3,Peaceful home,Private room in home,private_room,2023-07-31,2023-12-20,US,New Jersey,Stafford Township,8092.0,USD,...,59,306,32,65,7,4,92,92,3528,3528
4,Serenity over the bridge from LBI.,Private room in house,private_room,2020-07-12,2022-01-19,US,New Jersey,Stafford Township,8050.0,USD,...,0,365,0,60,0,10,140,140,168,168


In [50]:
sanFranciscoData.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,"Beach view, newly furnished 2 bdr apartment in SF",Entire rental unit,entire_home,2023-07-31,2023-12-13,US,California,San Francisco,94116.0,USD,...,99,266,64,78,1,-2,200,200,15574,15574
1,Ocean Beach gem w/ surf views,Entire rental unit,entire_home,2023-07-31,2024-01-10,US,California,San Francisco,94116.0,USD,...,344,21,267,93,40,37,306,306,103096,103096
2,Ocean Beach sky apartment,Entire rental unit,entire_home,2023-07-31,2023-10-21,US,California,San Francisco,94116.0,USD,...,110,255,101,100,9,6,252,252,27720,27720
3,Bright 2 Bedroom Home Walk To Beach,Entire home,entire_home,2023-07-31,2024-01-10,US,California,San Francisco,94116.0,USD,...,49,316,18,44,1,-2,141,141,3315,3315
4,San Francisco Beach House,Entire home,entire_home,2023-07-31,2024-01-10,US,California,San Francisco,94116.0,USD,...,108,257,67,74,1,-2,131,131,10920,10920


In [51]:
sydneyData.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,KUI COTTAGE private room with ensuite,Private room in home,private_room,2023-07-31,2024-01-10,AU,New South Wales,Camden,2570.0,AUD,...,61,304,17,33,10,7,78,117.0,1570,2355.0
1,River View Appartment (on Camden bike track),Entire rental unit,entire_home,2023-07-31,2024-01-10,AU,New South Wales,Camden,2570.0,AUD,...,351,14,219,75,34,31,93,139.5,26012,39018.0
2,Modern Luxury with Nature,Entire home,entire_home,2023-07-31,2024-01-10,AU,New South Wales,Spring Farm,2570.0,AUD,...,121,244,86,85,21,18,103,154.5,10594,15891.0
3,Tre Sorelle Camden,Entire rental unit,entire_home,2023-07-31,2024-01-10,AU,New South Wales,Camden,2570.0,AUD,...,287,78,164,69,41,38,180,270.0,35645,53467.5
4,Cozy 50s Cottage in Camden (our fibro Château :),Entire bungalow,entire_home,2023-07-31,2024-01-10,AU,New South Wales,Elderslie,2570.0,AUD,...,131,234,55,50,6,3,129,193.5,8804,13206.0


In [52]:
tokyoData.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Yamato Sta 1-min room A 203 (studio),Private room in rental unit,private_room,2023-07-31,2024-01-10,JP,Tokyo,Yamato shi,NaN,JPY,...,61,304,6,12,6,3,40,5831.6,653,95200.9
1,Yamato Sta 1-min room B 201 (studio),Private room in rental unit,private_room,2023-07-31,2024-01-10,JP,Tokyo,Yamato,NaN,JPY,...,206,159,106,62,12,9,41,5977.4,5957,868471.0
2,Yamato Sta 1-min room C 305 (long stay studio),Private room in rental unit,private_room,2023-07-31,2024-01-10,JP,Tokyo,Yamato-shi,NaN,JPY,...,246,119,74,36,6,3,38,5540.0,3725,543067.8
3,Fujino House / 藤野ハウス,Entire home,entire_home,2022-07-22,2024-01-10,JP,Tokyo,Sagamihara,NaN,JPY,...,286,79,211,89,57,46,64,9330.6,18514,2699156.1
4,【１棟貸し切りコンドミニアム 相模湖BASSHOUSE】,Entire bungalow,entire_home,2023-07-31,2024-01-10,JP,Tokyo,"Midori-ku, Sagamihara",NaN,JPY,...,327,38,193,71,78,75,203,29595.4,49393,7201005.5


In [16]:
torontoData.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Currency Native,Number of Reviews,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Waterfront Cozy Escape,Private room in rental unit,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,79,...,224,141,162,87,26,23,100,134.0,21568,28901.1
1,Live by the Lake Ontario-Entire Apartment,Entire condo,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,19,...,32,333,19,71,2,-1,102,136.7,2407,3225.4
2,Home away from home.,Private room in rental unit,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,5,...,173,192,73,51,3,0,74,99.2,6550,8777.0
3,"❤️Beautiful HOUSE!, near everything! LAKE+WIFI...",Entire home,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,50,...,78,287,33,51,5,2,65,87.1,3176,4255.8
4,💕BEAUTIFUL BSMT STUDIO By LAKE! + wifi PRKN & ...,Entire home,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,46,...,271,94,153,68,21,18,84,112.6,17958,24063.7


In [42]:
data = {}
for key, values in raw_data.items():
    data[key] = [values.shape[1], values.shape[0]]
data_summary = pd.DataFrame(data, index = ['columns', 'rows'])
print(data_summary)

         DubaiData  LosAngelesData  LondonData  MiamiData  NewYorkCityData  \
columns         53              54          54         57               54   
rows         13560           12837       43580      10205            40343   

         SanFranciscoData  SydneyData  TokyoData  TorontoData  
columns                54          54         54           53  
rows                 3801        5938       9661         5900  


In [23]:
#compare columns
unique_columns = set(miamiData.columns).intersection(
    set(dubaiData.columns), 
    set(losAngelesData.columns),
    set(londonData.columns),
    set(newYorkCityData.columns),
    set(sanFranciscoData.columns),
    set(sydneyData.columns),
    set(tokyoData.columns),
    set(torontoData.columns)
)
print(f'Number of columns in common: {len(unique_columns)}')

Number of columns in common: 52


In [65]:
print(f"miami_dubai_column_differences: {set(miamiData).difference(set(dubaiData.columns))}")
print(f"miami_toronto_column_differences: {set(miamiData).difference(set(torontoData.columns))}")
print(f"miami_la_column_differences: {set(miamiData).difference(set(losAngelesData.columns))}")
print(f"miami_london_column_differences: {set(miamiData).difference(set(londonData.columns))}")
print(f"miami_nyc_column_differences: {set(miamiData).difference(set(newYorkCityData.columns))}")
print(f"miami_sanfransisco_column_differences: {set(miamiData).difference(set(sanFranciscoData.columns))}")
print(f"miami_sydney_column_differences: {set(miamiData).difference(set(sydneyData.columns))}")
print(f"miami_tokyo_column_differences: {set(miamiData).difference(set(tokyoData.columns))}")

miami_dubai_column_differences: {'Neighbourhood', 'State', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_toronto_column_differences: {'Neighbourhood', 'Zipcode', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_la_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_london_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_nyc_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_sanfransisco_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_sydney_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_tokyo_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}


In [66]:
raw_data['miamiData']['Neighbourhood']

0                              NaN
1                    Coconut Grove
2        Miami Government District
3                         Brickell
4                  Design District
                   ...            
10200                          NaN
10201                          NaN
10202                          NaN
10203                          NaN
10204                          NaN
Name: Neighbourhood, Length: 10205, dtype: object

In [69]:
raw_data['miamiData']['Metropolitan Statistical Area']

0                                            NaN
1        Miami-Fort Lauderdale-Pompano Beach, FL
2        Miami-Fort Lauderdale-Pompano Beach, FL
3        Miami-Fort Lauderdale-Pompano Beach, FL
4        Miami-Fort Lauderdale-Pompano Beach, FL
                          ...                   
10200                                        NaN
10201                                        NaN
10202                                        NaN
10203                                        NaN
10204                                        NaN
Name: Metropolitan Statistical Area, Length: 10205, dtype: object

In [67]:
raw_data['miamiData']['Last Host Count Updated Date']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
10200    NaN
10201    NaN
10202    NaN
10203    NaN
10204    NaN
Name: Last Host Count Updated Date, Length: 10205, dtype: object

In [100]:
raw_data['miamiData'].drop(columns = ['Neighbourhood', 'Last Host Count Updated Date','Metropolitan Statistical Area'])

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Cozy Family home with a pool,Entire home,entire_home,2023-07-31,2024-01-03,US,NaN,Miami,33138,USD,...,84,281,6,9,0,-3,222,222,1678,1678
1,Coconut Grove Cottage in the Heart of the Vill...,Entire guesthouse,entire_home,2023-07-31,2023-10-11,US,Florida,Miami,33133,USD,...,178,187,83,56,21,18,94,94,11470,11470
2,DOWNTOWN MODERN INCREDIBLE VIEWS 2/2 LOFT,Entire rental unit,entire_home,2023-07-31,2024-01-03,US,Florida,Miami,33132,USD,...,92,273,70,91,15,12,291,291,25863,25863
3,"WELCOME TO PARADISE MIAMI, FLORIDA",Private room in rental unit,private_room,2023-07-31,2024-01-03,US,Florida,Miami,33129,USD,...,210,155,168,96,37,34,60,60,12096,12096
4,Garden Cottage- Heart of Design District,Entire guesthouse,entire_home,2023-07-31,2024-01-03,US,Florida,Miami,33137,USD,...,53,312,39,88,2,-1,52,52,2575,2575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10200,Charming Home in Buena Vista!,Entire home,entire_home,2023-09-26,2023-12-27,US,NaN,Miami,33137,USD,...,30,335,4,16,1,0,500,500,2400,2400
10201,Wyndwood Bungalow,Entire home,entire_home,2023-09-26,2023-12-27,US,NaN,Miami,33127,USD,...,31,334,24,93,0,-1,90,90,2595,2595
10202,Spectacular - 1B Apt in Downtown,Entire rental unit,entire_home,2023-09-26,2023-12-27,US,NaN,Miami,33132,USD,...,31,334,14,54,0,-1,220,220,3683,3683
10203,Amazing ocean & city views loft @brickell down...,Entire loft,entire_home,2023-09-26,2023-12-27,US,NaN,Miami,33131,USD,...,31,334,17,66,3,2,260,260,5320,5320


In [102]:
#add state column to dubai listing
raw_data['dubaiData']['state'] = 'dubai'

In [118]:
#drop last scraped data
def drop_last_scraped_data(df, df_name):
    df.drop(columns = ['Last Scraped Date'])
    print(f'{df_name} Last Scraped Date column dropped')

In [119]:
for df_name, df in raw_data.items():
    drop_last_scraped_data(df, df_name)

dubaiData Last Scraped Date column dropped
losAngelesData Last Scraped Date column dropped
londonData Last Scraped Date column dropped
miamiData Last Scraped Date column dropped
newYorkCityData Last Scraped Date column dropped
sanFranciscoData Last Scraped Date column dropped
sydneyData Last Scraped Date column dropped
tokyoData Last Scraped Date column dropped
torontoData Last Scraped Date column dropped


In [67]:
df_columns = ['Checkout Time', 'Minimum Stay', 'Max Guests', 'Airbnb Cleanliness Rating', 
    'Listing URL', 'Currency Native', 'Created Date', 'Instantbook Enabled', 
    'Average Daily Rate (USD)', 'Check-in Time', 'Cancellation Policy', 'picture_url', 
    'Annual Revenue LTM (USD)', 'Average Daily Rate (Native)', 'Exact Location', 'Airbnb Communication Rating',
    'Airbnb Property ID', 'Airbnb Accuracy Rating', 'Longitude', 'instant_bookable', 'Airbnb Location Rating', 
    'Amenities', 'Pets Allowed', 'Count Blocked Days LTM', 'Listing Type',
    'Number of Bookings LTM - Number of observed month', 'Latitude', 
    'Cleaning Fee (USD)', 'City', 'Extra People Fee (USD)', 'Overall Rating', 
    'Occupancy Rate LTM', 'Annual Revenue LTM (Native)', 'Host Listing Count', 'Bedrooms', 
    'Number of Reviews', 'Listing Title', 'Number of Bookings LTM', 'License', 'Airbnb Host ID', 
    'Count Available Days LTM', 'Bathrooms', 'Cleaning Fee (Native)', 'Airbnb Checkin Rating', 
    'Airbnb Superhost', 'Count Reservation Days LTM', 'Property Type', 
    'Country', 'Airbnb Value Rating', 'Extra People Fee(Native)', 'guest_controls']

In [96]:
#check na and null values
def detect_missing_values(df):
    is_na = {}
    for col in df_columns:
        empty = df[col] ==  ''
        null_values = df[col].isna().any()
        if null_values == True or empty.any() == True:
            is_na[col] = [df[col].isna().any(), df[col].isna().sum(), empty.sum()]
    print(f'{len(is_na)}\n')
    
    for key, value in is_na.items():
        print(f'{key}: {value}\n ')
    
    return is_na   

In [97]:
#apply detect values function
print(detect_missing_values(raw_data['losAngelesData']))

TypeError: 'NoneType' object is not subscriptable

In [72]:
print(detect_missing_values(raw_data['dubaiData']))

13

Checkout Time: [True, 1869, 0]
 
Airbnb Cleanliness Rating: [True, 3596, 0]
 
Check-in Time: [True, 644, 0]
 
Cancellation Policy: [True, 9161, 0]
 
Airbnb Communication Rating: [True, 3592, 0]
 
Airbnb Accuracy Rating: [True, 3592, 0]
 
instant_bookable: [True, 9161, 0]
 
Airbnb Location Rating: [True, 3592, 0]
 
Overall Rating: [True, 3614, 0]
 
Host Listing Count: [True, 8756, 0]
 
License: [True, 11602, 0]
 
Airbnb Checkin Rating: [True, 3592, 0]
 
Airbnb Value Rating: [True, 3592, 0]
 
{'Checkout Time': [True, 1869, 0], 'Airbnb Cleanliness Rating': [True, 3596, 0], 'Check-in Time': [True, 644, 0], 'Cancellation Policy': [True, 9161, 0], 'Airbnb Communication Rating': [True, 3592, 0], 'Airbnb Accuracy Rating': [True, 3592, 0], 'instant_bookable': [True, 9161, 0], 'Airbnb Location Rating': [True, 3592, 0], 'Overall Rating': [True, 3614, 0], 'Host Listing Count': [True, 8756, 0], 'License': [True, 11602, 0], 'Airbnb Checkin Rating': [True, 3592, 0], 'Airbnb Value Rating': [True, 

In [32]:
print(detect_missing_values(raw_data['londonData']))

14

Checkout Time: [True, 5320, 0]
 
Airbnb Cleanliness Rating: [True, 7385, 0]
 
Check-in Time: [True, 2616, 0]
 
Cancellation Policy: [True, 20839, 0]
 
Airbnb Communication Rating: [True, 6428, 0]
 
Airbnb Accuracy Rating: [True, 6429, 0]
 
instant_bookable: [True, 20838, 0]
 
Airbnb Location Rating: [True, 6429, 0]
 
Overall Rating: [True, 6476, 0]
 
Host Listing Count: [True, 19367, 0]
 
Listing Title: [True, 4, 0]
 
License: [True, 43580, 0]
 
Airbnb Checkin Rating: [True, 6430, 0]
 
Airbnb Value Rating: [True, 6429, 0]
 
{'Checkout Time': [True, 5320, 0], 'Airbnb Cleanliness Rating': [True, 7385, 0], 'Check-in Time': [True, 2616, 0], 'Cancellation Policy': [True, 20839, 0], 'Airbnb Communication Rating': [True, 6428, 0], 'Airbnb Accuracy Rating': [True, 6429, 0], 'instant_bookable': [True, 20838, 0], 'Airbnb Location Rating': [True, 6429, 0], 'Overall Rating': [True, 6476, 0], 'Host Listing Count': [True, 19367, 0], 'Listing Title': [True, 4, 0], 'License': [True, 43580, 0], 'Ai

In [33]:
print(detect_missing_values(raw_data['miamiData']))

17

Checkout Time: [True, 802, 0]
 
Airbnb Cleanliness Rating: [True, 1393, 0]
 
Check-in Time: [True, 324, 0]
 
Cancellation Policy: [True, 5745, 0]
 
Airbnb Communication Rating: [True, 1278, 0]
 
Airbnb Accuracy Rating: [True, 1278, 0]
 
instant_bookable: [True, 5745, 0]
 
Airbnb Location Rating: [True, 1278, 0]
 
Cleaning Fee (USD): [True, 5744, 0]
 
Extra People Fee (USD): [True, 5744, 0]
 
Overall Rating: [True, 2523, 0]
 
Host Listing Count: [True, 5339, 0]
 
License: [True, 10117, 0]
 
Cleaning Fee (Native): [True, 5744, 0]
 
Airbnb Checkin Rating: [True, 1278, 0]
 
Airbnb Value Rating: [True, 1278, 0]
 
Extra People Fee(Native): [True, 5744, 0]
 
{'Checkout Time': [True, 802, 0], 'Airbnb Cleanliness Rating': [True, 1393, 0], 'Check-in Time': [True, 324, 0], 'Cancellation Policy': [True, 5745, 0], 'Airbnb Communication Rating': [True, 1278, 0], 'Airbnb Accuracy Rating': [True, 1278, 0], 'instant_bookable': [True, 5745, 0], 'Airbnb Location Rating': [True, 1278, 0], 'Cleaning Fe

In [34]:
print(detect_missing_values(raw_data['newYorkCityData']))

13

Checkout Time: [True, 3952, 0]
 
Airbnb Cleanliness Rating: [True, 4833, 0]
 
Check-in Time: [True, 1922, 0]
 
Cancellation Policy: [True, 17276, 0]
 
Airbnb Communication Rating: [True, 4243, 0]
 
Airbnb Accuracy Rating: [True, 4243, 0]
 
instant_bookable: [True, 17274, 0]
 
Airbnb Location Rating: [True, 4243, 0]
 
Overall Rating: [True, 4281, 0]
 
Host Listing Count: [True, 15766, 0]
 
License: [True, 40222, 0]
 
Airbnb Checkin Rating: [True, 4243, 0]
 
Airbnb Value Rating: [True, 4244, 0]
 
{'Checkout Time': [True, 3952, 0], 'Airbnb Cleanliness Rating': [True, 4833, 0], 'Check-in Time': [True, 1922, 0], 'Cancellation Policy': [True, 17276, 0], 'Airbnb Communication Rating': [True, 4243, 0], 'Airbnb Accuracy Rating': [True, 4243, 0], 'instant_bookable': [True, 17274, 0], 'Airbnb Location Rating': [True, 4243, 0], 'Overall Rating': [True, 4281, 0], 'Host Listing Count': [True, 15766, 0], 'License': [True, 40222, 0], 'Airbnb Checkin Rating': [True, 4243, 0], 'Airbnb Value Rating':

In [73]:
print(detect_missing_values(raw_data['sanFranciscoData']))

13

Checkout Time: [True, 342, 0]
 
Airbnb Cleanliness Rating: [True, 271, 0]
 
Check-in Time: [True, 149, 0]
 
Cancellation Policy: [True, 687, 0]
 
Airbnb Communication Rating: [True, 254, 0]
 
Airbnb Accuracy Rating: [True, 254, 0]
 
instant_bookable: [True, 687, 0]
 
Airbnb Location Rating: [True, 255, 0]
 
Overall Rating: [True, 251, 0]
 
Host Listing Count: [True, 620, 0]
 
License: [True, 1183, 0]
 
Airbnb Checkin Rating: [True, 255, 0]
 
Airbnb Value Rating: [True, 255, 0]
 
{'Checkout Time': [True, 342, 0], 'Airbnb Cleanliness Rating': [True, 271, 0], 'Check-in Time': [True, 149, 0], 'Cancellation Policy': [True, 687, 0], 'Airbnb Communication Rating': [True, 254, 0], 'Airbnb Accuracy Rating': [True, 254, 0], 'instant_bookable': [True, 687, 0], 'Airbnb Location Rating': [True, 255, 0], 'Overall Rating': [True, 251, 0], 'Host Listing Count': [True, 620, 0], 'License': [True, 1183, 0], 'Airbnb Checkin Rating': [True, 255, 0], 'Airbnb Value Rating': [True, 255, 0]}


In [36]:
print(detect_missing_values(raw_data['sydneyData']))

14

Checkout Time: [True, 386, 0]
 
Airbnb Cleanliness Rating: [True, 582, 0]
 
Check-in Time: [True, 218, 0]
 
Cancellation Policy: [True, 910, 0]
 
Airbnb Communication Rating: [True, 539, 0]
 
Airbnb Accuracy Rating: [True, 539, 0]
 
instant_bookable: [True, 910, 0]
 
Airbnb Location Rating: [True, 539, 0]
 
Overall Rating: [True, 540, 0]
 
Host Listing Count: [True, 885, 0]
 
Listing Title: [True, 1, 0]
 
License: [True, 615, 0]
 
Airbnb Checkin Rating: [True, 539, 0]
 
Airbnb Value Rating: [True, 539, 0]
 
{'Checkout Time': [True, 386, 0], 'Airbnb Cleanliness Rating': [True, 582, 0], 'Check-in Time': [True, 218, 0], 'Cancellation Policy': [True, 910, 0], 'Airbnb Communication Rating': [True, 539, 0], 'Airbnb Accuracy Rating': [True, 539, 0], 'instant_bookable': [True, 910, 0], 'Airbnb Location Rating': [True, 539, 0], 'Overall Rating': [True, 540, 0], 'Host Listing Count': [True, 885, 0], 'Listing Title': [True, 1, 0], 'License': [True, 615, 0], 'Airbnb Checkin Rating': [True, 539

In [37]:
print(detect_missing_values(raw_data['tokyoData']))

12

Checkout Time: [True, 311, 0]
 
Airbnb Cleanliness Rating: [True, 1204, 0]
 
Check-in Time: [True, 123, 0]
 
Cancellation Policy: [True, 2935, 0]
 
Airbnb Communication Rating: [True, 1029, 0]
 
Airbnb Accuracy Rating: [True, 1029, 0]
 
instant_bookable: [True, 2935, 0]
 
Airbnb Location Rating: [True, 1029, 0]
 
Overall Rating: [True, 1033, 0]
 
Host Listing Count: [True, 2802, 0]
 
Airbnb Checkin Rating: [True, 1029, 0]
 
Airbnb Value Rating: [True, 1029, 0]
 
{'Checkout Time': [True, 311, 0], 'Airbnb Cleanliness Rating': [True, 1204, 0], 'Check-in Time': [True, 123, 0], 'Cancellation Policy': [True, 2935, 0], 'Airbnb Communication Rating': [True, 1029, 0], 'Airbnb Accuracy Rating': [True, 1029, 0], 'instant_bookable': [True, 2935, 0], 'Airbnb Location Rating': [True, 1029, 0], 'Overall Rating': [True, 1033, 0], 'Host Listing Count': [True, 2802, 0], 'Airbnb Checkin Rating': [True, 1029, 0], 'Airbnb Value Rating': [True, 1029, 0]}


In [38]:
print(detect_missing_values(raw_data['torontoData']))

14

Checkout Time: [True, 471, 0]
 
Airbnb Cleanliness Rating: [True, 606, 0]
 
Check-in Time: [True, 264, 0]
 
Cancellation Policy: [True, 1515, 0]
 
Airbnb Communication Rating: [True, 552, 0]
 
Airbnb Accuracy Rating: [True, 552, 0]
 
instant_bookable: [True, 1515, 0]
 
Airbnb Location Rating: [True, 552, 0]
 
Overall Rating: [True, 557, 0]
 
Host Listing Count: [True, 1444, 0]
 
Listing Title: [True, 1, 0]
 
License: [True, 2437, 0]
 
Airbnb Checkin Rating: [True, 552, 0]
 
Airbnb Value Rating: [True, 552, 0]
 
{'Checkout Time': [True, 471, 0], 'Airbnb Cleanliness Rating': [True, 606, 0], 'Check-in Time': [True, 264, 0], 'Cancellation Policy': [True, 1515, 0], 'Airbnb Communication Rating': [True, 552, 0], 'Airbnb Accuracy Rating': [True, 552, 0], 'instant_bookable': [True, 1515, 0], 'Airbnb Location Rating': [True, 552, 0], 'Overall Rating': [True, 557, 0], 'Host Listing Count': [True, 1444, 0], 'Listing Title': [True, 1, 0], 'License': [True, 2437, 0], 'Airbnb Checkin Rating': [T

In [128]:
#remove emoji characters
def remove_emoji(df, df_name):
    emoji_patterns = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U00002B00-\U00002BFF"  # Miscellaneous Symbols and Arrows
        u"\U0001F100-\U0001F1FF"  # Enclosed Alphanumeric Supplement
        u"\U0001F200-\U0001F2FF"  # Enclosed Ideographic Supplement
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u" \U000025A0-\U000025FF" # Geometric Shapes
         "]+", flags = re.UNICODE)
    
    df = df.map(lambda x: emoji_patterns.sub(r' ', x) if isinstance(x, str) else x)
    print(f'All emojis removed from {df_name}')

    return df


In [129]:
for df_name, df in raw_data.items():
    remove_emoji(df, df_name)

All emojis removed from dubaiData
All emojis removed from losAngelesData
All emojis removed from londonData
All emojis removed from miamiData
All emojis removed from newYorkCityData
All emojis removed from sanFranciscoData
All emojis removed from sydneyData
All emojis removed from tokyoData
All emojis removed from torontoData


In [122]:
#generate dimension tables id
def generate_ids(df, df_acroynm, dim_acroynm):
    size = len(df)
    random_numbers = np.random.randint(low = 10000, high = 90000, size = size)
    ids = [df_acroynm + dim_acroynm + str(number) for number in random_numbers]
    return ids

def add_id_columns(df,  df_acroynm):
    ids = {'listingID':'LIS', 'amenitiesID':'AMT', 'ratingID':'RTE', 'bookingID':'BKD'}
    for id in ids:
       df[id] = generate_ids(df, df_acroynm, ids[id])
    print(f'Ids created sucessfully')
  

In [123]:
for df_name, df in raw_data.items():
    add_id_columns(df, df_name[:3].upper())

Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully
Ids created sucessfully


In [134]:
def translate_row(row):
    translation = GoogleTranslator(source = 'auto', target='en').translate(row)
    return translation

In [135]:
def detect_lang(row):
    try:
        detect(row)
    except langdetect.lang_detect_exception.LangDetectException:
        pass

In [137]:
def replace_non_english(df_name, df):
    def replace_cell(value):
        if pd.isna(value) == True:
            return value
        if not isinstance(value, str):
            return value
        if detect_lang(value) != 'en':
            translation = translate_row(value)
            return translation
        else:
            return value
    print(f'{df_name} values translated')

    return  df.map(replace_cell)

In [138]:
for df_name, df in raw_data.items():
    replace_non_english(df_name, df)

dubaiData values translated


In [ ]:
            if detect_lang(translation) != 'en':
                for loc in ['City', 'Zipcode', 'State']:
                    if pd.notnull(df[loc]) and pd.notnull(df['Bedrooms']):
                        df['Listing Title'] = str(df.loc['Bedrooms']) + ' ' + str(df.at[idx, loc])
                        break

In [ ]:
def create_dimension_fact_tables(df):
    listing_dim = df[['listingID','Listing Title', 'Property Type', 'Listing Type', 'Listing URL', 'Created Date', 'Currency Native']]
    location_dim = df[['Airbnb Property ID', 'country', 'state', 'city', 'Zipcode', 'latitude', 'longtitude', 'Exact Location']]
    host_dim = df[['Airbnb Host ID', 'Host Listing Count']]
    amenities_dim = df[['amenitiesID', 'amenities']]
    rating_dim = df[['ratingID', 'Overall Rating', 'Airbnb Communication Rating' ,'Airbnb Accuracy Rating'
                     , 'Airbnb Cleanliness Rating', 'Airbnb Checkin Rating', 'Airbnb Location Rating', 'Airbnb Value Rating']]
    booking_dim = df[['bookingID', 'Check-in Time', 'Checkout Time', 'Minimum Stay']]
    booking_fact = df[['bookingID', 'listingID', 'Airbnb Host ID', 'Airbnb Property ID', 'amenitiesID',
                    'ratingID', 'Number of Reviews', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Airbnb Superhost',
                    'Cancellation Policy', 'Cleaning Fee (USD)', 'Cleaning Fee (Native)', 'Extra People Fee (USD)',
                    'Extra People Fee (Native)', 'instant_bookable', 'Pets Allowed', 'Occupancy Rate LTM',
                    'Number of Bookings LTM', 'Number of Observed Months', 'Average Daily Rate (USD)', 'Annual Revenue LTM (USD)']]
    

In [ ]:
#rename lisitng_dim columns
listing_dim.rename(columns = {'Listing Title': 'listingTitle',
                              'Property Type': 'propertyType',
                              'Listing Type': 'listingType',
                              'Listing URL': 'listingUrl',
                              'Created Date': 'createdDate',
                              'Currency Native': 'currencyNative'
                        })

In [ ]:
#rename location_dim columns
location_dim.rename(columns ={'Airbnb Property ID':'propertyID','Exact Location':'exactLocation' })

In [ ]:
#rename host_dim columns
host_dim.rename(columns = {'Airbnb Host ID': 'hostID', 'Host Listing Count': 'hostListingCount'})

In [ ]:
#rename rating_dim columns
rating_dim.rename(columns = {'Overall Rating':'overallRating', 'Airbnb Communication Rating': 'communicationRating', 
                             'Airbnb Accuracy Rating': 'accuracyRating', 'Airbnb Cleanliness Rating': 'cleanlinessRating', 
                             'Airbnb Checkin Rating': 'checkinRating', 'Airbnb Location Rating': 'locationRating', 'Airbnb Value Rating': 'valueRating'
                                       })

In [ ]:
#rename booking_dim columns
booking_dim.rename(columns = {'Check-in Time': 'checkInTime', 'Checkout Time': 'checkoutTime', 'Minimum Stay': 'minimumStay'})

In [ ]:
#rename booking_dim columns
booking_fact.rename(columns = {
                            'Airbnb Host ID': 'hostID', 'Airbnb Property ID': 'propertyID', 
                            'Number of Reviews': 'numberOfReviews', 'Bedrooms': 'bedrooms', 'Bathrooms': 'bathrooms',
                            'Max Guests': 'maxGuests', 'Airbnb Superhost': 'airbnbSuperhost', 'Cancellation Policy': 'cancellationPolicy', 
                            'Cleaning Fee (USD)': 'cleaningFee(USD)', 'Cleaning Fee (Native)': 'cleaningFee(Native)', 
                            'Extra People Fee (USD)': 'extraPeopleFee(USD)', 'Extra People Fee (Native)': 'extraPeopleFee(Native)', 'instant_bookable': 'instantBookable', 
                            'Pets Allowed': 'petsAllowed', 'Occupancy Rate LTM': 'occupancyRateLTM','Number of Bookings LTM': 'numberOfBookingsLTM', 
                            'Number of Observed Months': 'numberOfObservedMonths', 'Average Daily Rate (USD)':'averageDailyRate(USD)', 
                            'Annual Revenue LTM (USD)': 'annualRevenueLTM(USD)'

                                    })

In [130]:
print(raw_data['miamiData'].columns)

Index(['Listing Title', 'Property Type', 'Listing Type', 'Created Date',
       'Last Scraped Date', 'Last Host Count Updated Date', 'Country', 'State',
       'City', 'Zipcode', 'Neighbourhood', 'Metropolitan Statistical Area',
       'Currency Native', 'Number of Reviews', 'Bedrooms', 'Bathrooms',
       'Max Guests', 'Airbnb Superhost', 'Cancellation Policy',
       'Cleaning Fee (USD)', 'Cleaning Fee (Native)', 'Extra People Fee (USD)',
       'Extra People Fee(Native)', 'Check-in Time', 'Checkout Time',
       'Minimum Stay', 'Latitude', 'Longitude', 'Exact Location',
       'Overall Rating', 'Airbnb Communication Rating',
       'Airbnb Accuracy Rating', 'Airbnb Cleanliness Rating',
       'Airbnb Checkin Rating', 'Airbnb Location Rating',
       'Airbnb Value Rating', 'Amenities', 'picture_url', 'License',
       'Airbnb Property ID', 'Airbnb Host ID', 'Host Listing Count',
       'guest_controls', 'instant_bookable', 'Pets Allowed', 'Listing URL',
       'Instantbook Enabled', 

In [132]:
%store -z#